## Imports for Evaluation

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import ipywidgets as widgets
from IPython.display import display
from pathlib import Path
from datetime import datetime
from revcan.reverse_engineering.models.experiment import Experiment
from revcan.reverse_engineering.models.solutions import Solutions
import matplotlib.pyplot as plt
import matplotlib as mpl

### Analyze Experiment

Set path to combined experiment folder

In [ ]:

experiment_folder = "../../data/experiments/car/accelerator_pedal_position"

## Start analysis

Python Interactive mode used in Jupyter Notebooks can sometimes cause problems with multiprocess applications.

If you can't execute the cell below, try to restart the kernel, use the commented line or rune the line in a command line.

The analysis can run for multiple hours

In [ ]:

#!python -u ../scripts_for_doip_new/experiment_analysis_parallel.py --experiment_file_path "{Path(experiment_folder).absolute()}/combined.json" --output_file_path "{Path(experiment_folder).absolute()}/solution.json"
%run ../scripts_for_doip_new/experiment_analysis_parallel.py --experiment_file_path "{Path(experiment_folder).absolute()}/combined.json" --output_file_path "{Path(experiment_folder).absolute()}/solution.json"

Load results from analysis

In [ ]:

solutions = Solutions.load(f"{Path(experiment_folder).absolute()}/solution.json")

Sort results by residuals

In [ ]:
sorted_solutions = sorted(solutions.solutions, key = lambda s:s.residuals[0])

In [ ]:


#TODO set following values
n_signals_to_plot=1 #number of traces to plot
ylabel = "Accelerator pedal position in %" # y axis label
output_file_name = "Pedal_position"

mpl.rcParams['font.family']='Times New Roman'
plt.figure(figsize=(6, 3))

x_gt = list(range(0,len(sorted_solutions[0].y)))
y_gt = []
for i in range(0,len(solutions.groundtruth.values),int (len(solutions.groundtruth.values)/len(sorted_solutions[0].y))):
    y_gt.append(solutions.groundtruth.values[i].value[0])

plt.plot(x_gt,y_gt, label='Ground truth', marker='x')


for i in range(0,n_signals_to_plot):
    plt.plot(list(range(0,len(sorted_solutions[0].y))), sorted_solutions[i].x*sorted_solutions[i].coefficients[0]+sorted_solutions[i].coefficients[1], label=f'Best match {i+1}', marker='o')
    print(sorted_solutions[i])


plt.xlabel("Sample number")
plt.ylabel(ylabel)
plt.legend()
plt.grid(True)
plt.xticks(range(0, max(x_gt) + 1))
plt.tight_layout()
plt.savefig(f"outputs/{output_file_name}.pdf")
plt.savefig(f"outputs/{output_file_name}.svg")
plt.savefig(f"outputs/{output_file_name}.eps")
plt.show()